In [16]:
import numpy as np # 
import pandas as pd

df_pha = pd.read_csv('datasets/Psychosocial_Health_Analysis.csv')
df_pha['problem_category'] = df_pha['problem_category'].str.lower().str.strip()
df_pha['psychological_catehory'] = df_pha['psychological_catehory'].str.lower().str.strip()
df_pha['problem_summary'] = df_pha['problem_summary'].str.lower().str.strip()

df_pha = df_pha[
    (df_pha['problem_category'] == 'stress') | 
    (df_pha['problem_category'] == 'anxiety') |
    (df_pha['psychological_catehory'] == 'stress') |
    (df_pha['psychological_catehory'] == 'anxiety')
    ]

df_pha.head(200)

,Age,Gender,Problem_description,problem_summary,problem_category,psychological_catehory
0,28,Female,A woman is sharing their challenges as the on...,financial and emotional stress seeking support.,financial,stress
3,20,Male,"How to increase confidence? Feeling insecure, ...",seeking guidance on improving confidence and d...,self-esteem,stress
5,22,Female,"The user is in a complicated situation, caught...",relationship dilemma: committed boyfriend vs n...,relationship,anxiety
6,25,Female,The user was previously in a relationship but ...,struggling to move on from past relationship.,relationship,stress
8,24,Male,The user is experiencing emotional turmoil due...,neglect in a long-term relationship causes dis...,relationship,stress
9,23,Female,The user feels overwhelmed by their spouse's e...,user seeks help dealing with spouse's demands,relationship,stress
13,24,Male,"The user is troubled by financial constraints,...",user faces legal and familial pressure hinderi...,legal,anxiety
16,24,Female,A widow faces pressure from her late husband's...,widow pressured about daughter's birth registr...,family,stress
24,17,Male,The individual faces academic challenges due ...,academic hurdles due to physical constraints,educational,stress
26,24,Female,"After getting married, the person is facing v...",financial strain and in-law demands affecting ...,marital,stress
